# AIリテラシ　第７回目
### AIによる画像認識 -1-
___
__手書き文字の認識__  
MNISTデータセットによる手書き文字の認識

### 必要なモジュールのインポート

In [ ]:
import numpy as np
from numpy.random import multivariate_normal, permutation
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.python.keras import layers, models

print("TensolFlow : {}".format(tf.__version__))
print("Keras : {}".format(tf.keras.__version__))
print("ExecutingEager : {}".format(tf.executing_eagerly()))

In [ ]:
!pip install japanize-matplotlib

In [ ]:
import japanize_matplotlib

#### 学習データの準備

In [ ]:
mnist = tf.keras.datasets.mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((len(train_images), 784)).astype('float32') / 255
test_images = test_images.reshape((len(test_images), 784)).astype('float32') / 255
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

### 学習データの確認

In [ ]:
train_images[0]

In [ ]:
train_labels[0]

先頭を0とした時に、7の位置のみが1となっている。<br>
「k番目の要素のみが1となっているリスト」でワンホット・エンコーディングと呼ぶ<br>
機械学習でデータをいくつかのグループに分類する時によく使用される。

### 画像イメージの確認

In [ ]:
fig = plt.figure(figsize=(8, 4))
for c, (image, label) in enumerate(zip(train_images[:10], train_labels[:10])):
    subplot = fig.add_subplot(2, 5, c+1)
    subplot.set_xticks([])
    subplot.set_yticks([])
    subplot.set_title('%d' % np.argmax(label))
    subplot.imshow(image.reshape((28, 28)), vmin=0, vmax=1, cmap=plt.cm.gray_r)

In [ ]:
for i in [0,4,5,9]:
    print("train_labels", "(i="+str(i)+"): ", train_labels[i])
    print("train_images", "(i="+str(i)+"): ")    
    plt.imshow(train_images[i].reshape([28, 28]), cmap='gray')
    plt.show()

### モデルの定義

In [ ]:
model = models.Sequential()
model.add(layers.Dense(10, activation='softmax', input_shape=(28*28,), name='softmax'))
model.summary()


### モデルのコンパイル

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

### 学習処理

In [ ]:
history = model.fit(train_images, train_labels,
                    validation_data=(test_images, test_labels),
                    batch_size=128, epochs=10, verbose=1)

### 学習済みモデルによる予測結果の確認

In [ ]:
p_val = model.predict(np.array(test_images))
df = DataFrame({'pred': list(map(np.argmax, p_val)),
                'label': list(map(np.argmax, test_labels))})
correct = df[df['pred']==df['label']]
incorrect = df[df['pred']!=df['label']]

fig = plt.figure(figsize=(8, 15))
plt.subplots_adjust(wspace=0.4, hspace=0.6) # subplot間の余白を設定
for i in range(10):
    indices = list(correct[correct['pred']==i].index[:3]) + list(incorrect[incorrect['pred']==i].index[:3])
    for c, image in enumerate(test_images[indices]):
        subplot = fig.add_subplot(10, 6, i*6+c+1)
        subplot.set_xticks([])
        subplot.set_yticks([])
        subplot.set_title('%d / %d' % (i, df['label'][indices[c]]))
        subplot.imshow(image.reshape((28, 28)), vmin=0, vmax=1, cmap=plt.cm.gray_r)